# Exploratory Data Analytics
---
### Loading the dataset
We start by loading the dataset from the CSV file using pandas.
This file contains the historical daily prices of the S&P 500 index.

In [4]:
import pandas as pd
df = pd.read_csv("../data/SPX.csv")

---

### Missing values & Duplicated rows


In [33]:
print("Missing values per column:\n", df.isnull().sum())

print("Number of duplicated rows:", df.duplicated().sum())

Missing values per column:
 Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64
Number of duplicated rows: 0


The dataset contains no missing values or duplicated rows, which means the data is complete and consistent.
We can proceed without applying any cleaning operations for these aspects.

---

### Basic DataFrame information & descriptive statistics 

In [34]:
print(df.info())

print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23323 entries, 0 to 23322
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       23323 non-null  object 
 1   Open       23323 non-null  float64
 2   High       23323 non-null  float64
 3   Low        23323 non-null  float64
 4   Close      23323 non-null  float64
 5   Adj Close  23323 non-null  float64
 6   Volume     23323 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.2+ MB
None
               Open          High           Low         Close     Adj Close  \
count  23323.000000  23323.000000  23323.000000  23323.000000  23323.000000   
mean     486.821737    489.690119    483.762812    486.918598    486.918598   
std      726.187716    730.112168    721.890344    726.278798    726.278798   
min        4.400000      4.400000      4.400000      4.400000      4.400000   
25%       23.855001     23.855001     23.855001     23.855001     23.8